In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}




Preconfiguring packages ...
Selecting previously unselected package cron.
(Reading database ... 18408 files and directories currently installed.)
Preparing to unpack .../00-cron_3.0pl1-128ubuntu5_amd64.deb ...
Unpacking cron (3.0pl1-128ubuntu5) ...
Selecting previously unselected package libapparmor1:amd64.
Preparing to unpack .../01-libapparmor1_2.11.0-2ubuntu17.1_amd64.deb ...
Unpacking libapparmor1:amd64 (2.11.0-2ubuntu17.1) ...
Selecting previously unselected package libdbus-1-3:amd64.
Preparing to unpack .../02-libdbus-1-3_1.10.22-1ubuntu1_amd64.deb ...
Unpacking libdbus-1-3:amd64 (1.10.22-1ubuntu1) ...
Selecting previously unselected package dbus.
Preparing to unpack .../03-dbus_1.10.22-1ubuntu1_amd64.deb ...
Unpacking dbus (1.10.22-1ubuntu1) ...
Selecting previously unselected package dirmngr.
Preparing to unpack .../04-dirmngr_2.1.15-1ubuntu8.1_amd64.deb ...
Unpacking dirmngr (2.1.15-1ubuntu8.1) ...
Selecting previously unselected package distro-info-data.
Preparing to unpack .

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

fuse: mountpoint is not empty
fuse: if you are sure this is safe, use the 'nonempty' mount option


In [0]:
cd drive/project_data/final_pascal_npy_datas

/content/drive/project_data/final_pascal_npy_datas


In [0]:
pwd

'/content/drive/project_data/final_pascal_npy_datas'

In [0]:
import keras
from keras.models import Sequential,load_model
from keras import layers, Model,Input
from keras.layers.normalization import BatchNormalization
from keras import optimizers
import numpy as np
import tensorflow as tf
from keras.optimizers import Adam
from keras import backend as K
from keras.layers.merge import concatenate
from keras.layers.advanced_activations import LeakyReLU
from keras.preprocessing import image

Using TensorFlow backend.


In [0]:
boxes = 3
classes = 20
true_boxes  = Input(shape=(9,9,3,4))

In [0]:
img_input = Input(shape=(300, 300, 3))

# VGG-16
x = layers.Conv2D(64, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block1_conv1',
                  trainable=False)(img_input)
x = layers.Conv2D(64, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block1_conv2',
                  trainable=False)(x)
x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

# Block 2
x = layers.Conv2D(128, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block2_conv1',
                  trainable=False)(x)
x = layers.Conv2D(128, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block2_conv2',
                  trainable=False)(x)
x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

# Block 3
x = layers.Conv2D(256, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block3_conv1',
                  trainable=False)(x)
x = layers.Conv2D(256, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block3_conv2',
                  trainable=False)(x)
x = layers.Conv2D(256, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block3_conv3',
                  trainable=False)(x)
x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

# Block 4
x = layers.Conv2D(512, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block4_conv1',
                  trainable=False)(x)
x = layers.Conv2D(512, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block4_conv2',
                  trainable=False)(x)
x = layers.Conv2D(512, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block4_conv3',
                  trainable=False)(x)
x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

# Block 5
x = layers.Conv2D(512, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block5_conv1',
                  trainable=False)(x)
x = layers.Conv2D(512, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block5_conv2',
                  trainable=False)(x)
x = layers.Conv2D(512, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block5_conv3',
                  trainable=False)(x)
classifier_output = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)
# VGG-Output
#skip connection
bypass = classifier_output

#detector block 1
output = layers.Conv2D(512, (1, 1),
                  activation='relu',
                  padding='same',
                  name='detector_block_1_1')(classifier_output)
output = layers.Conv2D(512, (1, 1),
                  activation='relu',
                  padding='same',
                  name='detector_block_1_2')(output)
#Merging
output = concatenate([bypass,output])
#detector block 2
output = layers.Conv2D(512, (2, 2),
                  activation='relu',
                  padding='same',
                  name='detector_block_2_1')(output)
output = layers.Conv2D(256, (2, 2),
                  activation='relu',
                  padding='same',
                  name='detector_block_2_2')(output)

## Comment the class= 0 for classification along with detection

classes = 0
output = BatchNormalization(name='detector_block_2_norm')(output)
output = LeakyReLU(alpha=0.1)(output)
output = layers.Conv2D(boxes * (4 + 1 + classes),(1,1),
                padding='same',
                name='detector_block_2_3')(output)
output = layers.Reshape((9, 9, boxes, 4 + 1 + classes))(output)

output = layers.Lambda(lambda args: args[0])([output, true_boxes])

output = layers.Dense(5,name = "Fc_after_1")(output)

output = layers.Reshape((9, 9, 15))(output)

output = layers.Dense(5,name = "Fc_after_2")(output)

model = Model([img_input, true_boxes], output)

In [0]:
def custom_loss(y_true,y_pred):
    loss = tf.reduce_mean(y_true[0,:,0:,0:1]*tf.squared_difference(y_true[0,:,0:,1:5],y_pred[0,:,0:,1:5]))
    conf_loss = tf.reduce_mean(tf.squared_difference(y_true[0,:,0:,0:1],y_pred[0,:,0:,0:1]))
    return loss+conf_loss
# def custom_acc(y_true,y_pred):
#     loss = y_true[0,:,0:,0:1]*tf.squared_difference(y_true[0,:,0:,1:5],y_pred[0,:,0:,1:5])
#     conf_loss = tf.squared_difference(y_true[0,:,0:,0:1],y_pred[0,:,0:,0:1])
    
    
#model = Model([img_input, true_boxes], output)

In [0]:
model.load_weights('classifier_weights.h5',by_name=True)
optimizer = Adam(lr=0.5e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss=custom_loss, optimizer=optimizer,metrics=['accuracy'])

In [0]:
#model.summary()

In [0]:
#loading data
X = np.load('X/2007_test.npy')
Y = np.load('Y/2007_test_labels.npy')
box_ = np.load('default_all_boxes_new.npy')

In [0]:
def bxx(box,sh):
    boxx = np.zeros(shape=(sh,9,9,3,4))
    for i in range(sh):
        boxx[i]=box
    return boxx
    

In [0]:
train_X = X[0:4500]
train_Y = Y[0:4500]
test_X = X[4500:]
test_Y = Y[4500:]
box_train = bxx(box_,train_X.shape[0])
box_test = bxx(box_,test_X.shape[0])

In [0]:
print("train x: ",train_X.shape)
print("train y: ",train_Y.shape)
print("test x: ",test_X.shape)
print("test y: ",test_Y.shape)
print("boxes train: ",box_train.shape)
print("boxes test: ",box_test.shape)

train x:  (4500, 300, 300, 3)
train y:  (4500, 9, 9, 5)
test x:  (452, 300, 300, 3)
test y:  (452, 9, 9, 5)
boxes train:  (4500, 9, 9, 3, 4)
boxes test:  (452, 9, 9, 3, 4)


In [0]:
history0 = model.fit([train_X,box_train],
                      train_Y,
                      batch_size=1,
                      epochs=1,
                      validation_data=([test_X,box_test],test_Y),
                      verbose=1)

In [0]:
#model.fit([train_X,box_],train_Y,epochs=5,validation_data=([test_X,box_],test_Y),verbose=1)

history1 = model.fit([train_X,box_train],
                      train_Y,
                      batch_size=16,
                      epochs=10,
                      validation_data=([test_X,box_test],test_Y),
                      verbose=1)

Train on 4500 samples, validate on 452 samples
Epoch 1/10
4500/4500 [==============================] - 101s 22ms/step - loss: 668.3646 - acc: 0.1138 - val_loss: 685.0265 - val_acc: 0.2097
Epoch 2/10
4500/4500 [==============================] - 100s 22ms/step - loss: 614.2833 - acc: 0.2076 - val_loss: 626.9009 - val_acc: 0.2748
Epoch 3/10
4500/4500 [==============================] - 100s 22ms/step - loss: 599.7597 - acc: 0.3583 - val_loss: 512.0421 - val_acc: 0.2982
Epoch 4/10
4500/4500 [==============================] - 100s 22ms/step - loss: 521.4988 - acc: 0.4571 - val_loss: 517.5743 - val_acc: 0.4768
Epoch 5/10
4500/4500 [==============================] - 100s 22ms/step - loss: 446.1575 - acc: 0.5001 - val_loss: 453.6654 - val_acc: 0.4834
Epoch 6/10
 304/4500 [=>............................] - ETA: 1:25 - loss: 400.5648 - acc: 0.4996

In [0]:
history2 = model.fit([train_X,box_train],
            train_Y,
            batch_size=32,
            epochs=10,
            validation_data=([test_X,box_test],test_Y),
            verbose=1)

(243, 4)

(9, 9, 3, 4)